In [ ]:
from forward import solve_forward
from simulation import Simulation
import numpy as np
import mat73
#from net import EEG_CNN

In [ ]:
fwd = solve_forward(downsampled_dipoles=False)
# eeg_topos = mat73.loadmat('../../Downloads/eeg_topos.mat')['eeg_topos']
# eeg_topos = eeg_topos.transpose(2, 0, 1)
# sources = np.load('../../Downloads/sources.npy')
# eeg = np.load('../../Downloads/eeg.npy')
# sim = Simulation(fwd,source_data=sources,eeg_data=eeg)
# sim.simulate(n_samples=100)

In [ ]:
# for ii in range(100):
#     np.save('../../Downloads/sources/regr/source_{}.npy'.format(ii+1), sim.source_data[:,ii])

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import (Dense, Dropout, Conv2D, Flatten, MaxPooling2D, Conv1D, BatchNormalization) 

In [ ]:
import keras_preprocessing_custom.image
loader = keras_preprocessing_custom.image.DataLoader()

In [ ]:
train_loader = loader.flow_from_directory(
    directory_y='../../Downloads/sim/source_data/',
    directory_x='../../Downloads/sim/topo_data/',
    batch_size=32,
)

In [ ]:
n_dipoles = fwd.leadfield.shape[1]
model = keras.Sequential()

# add input layer
model.add(keras.Input(shape=(67,67,1), name='Input'))
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu'))
#self.model.add(BatchNormalization())
model.add(Flatten())            
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
# Add output layer
model.add(Dense(n_dipoles, activation='relu', name='OutputLayer'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

        
model.compile(optimizer, 'mse', metrics=None)

In [ ]:
topos_eval = mat73.loadmat('../../Downloads/valid/eeg_topos.mat')['eeg_topos']
topos_eval = topos_eval.transpose(2, 0, 1)[0:32]

sources_eval = np.load('../../Downloads/valid/sources.npy').T
model.fit(x=train_loader,batch_size=32,epochs=4,validation_data=(topos_eval,sources_eval),shuffle=False)